In [1]:
## all imports
from IPython.display import HTML
import numpy as np
import urllib2
import bs4 #this is beautiful soup
import time
import operator
import socket
import cPickle
import re # regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

In [275]:
# Fixed url for job postings containing data scientist
url = 'http://www.indeed.com/jobs?q=data+scientist&l='
# read the website
source = urllib2.urlopen(url).read()
# parse html code
bs_tree = bs4.BeautifulSoup(source, 'html.parser')

In [276]:
job_count_string = bs_tree.find(id = 'searchCount').contents[0]
#print job_count_string
#Jobs 1 to 50 of 21,383

#print job_count_string.split()
#[u'Jobs', u'1', u'to', u'50', u'of', u'21,383']

#print job_count_string.split()[-1]
#21,383

#print type(job_count_string.split())
#<type 'list'>

jc = job_count_string.split()[-1]
jc = jc.replace(",", "")
print 'There are ' + jc + ' Data Scientist job openings'
jc = int(jc)

There are 21343 Data Scientist job openings


In [387]:
# The website is only listing 10 results per page, 
# so we need to scrape them page after page
num_pages = int(np.ceil(jc/10.0))

# Lambda expressions return the value of the expression inside it.
# In this case, it will return a string with new line characters replaced by spaces.
rem_nl = lambda s: s.replace("\n", " ")

base_url = 'http://www.indeed.com'
job_links = []
location = [];
company_name = [];
job_title = [];
for i in range(15): #do range(num_pages) if you want them all
    print 'Scraped page ' + str(i+1)
    url = base_url + '/jobs?q=data+scientist&start=' + str(i*10)
    html_page = urllib2.urlopen(url).read() 
    bs_tree = bs4.BeautifulSoup(html_page)
    
    # id = resultsCol is the attribute where the search results are
    job_link_area = bs_tree.find(id = 'resultsCol')
    
    # extracting all the job postings and putting them into the list when they are of class row result.
    # The reason it is done this way is because some of them are of class "row result" and others of class
    # " row result" with a whitespace, making it weird for beautiful soup. 
    job_postings = job_link_area.findAll("div")
    job_postings = [jp for jp in job_postings if not jp.get('class') is None 
                     and ''.join(jp.get('class')) =="rowresult"]
    
    # extract the job ids in the posting which have attribute data-jk
    job_ids = [jp.get('data-jk') for jp in job_postings];

    # extracting location which has heading span and attribute location. Need to check if any of them have weird
    # whitespaces, in which case the above method might be better
    # get text gives us the text inside <span class = location
    locations_this_page = [str(jp.find("span","location").get_text()) for jp in job_postings];
    
    # Same thing for company names, except get_text gives us something with \n new lines and white spaces. 
    # So we use the rem_nl method to remove the new lines and the strip() to remove white spaces
    company_names_this_page = [str(rem_nl(jp.find("span",{"company","a"}).get_text())).strip() for jp in job_postings]    
    
    # go after each link
    for id in job_ids:
        job_links.append(base_url + '/rc/clk?jk=' + id)
    # Append all the job locations from this page
    for loc in locations_this_page:
        location.append(loc)
    # Append all the company names from this page
    for cn in company_names_this_page:
        company_name.append(cn)

# Was planning to account for all cases, but there are too many

# for i in xrange(len(location)):
#     if location[i] == 'Remote':
#         location[i] = 'Remote, NS'
#     elif location[i] == 'United States':
#         location[i] = 'USA, NS'

# So if the format is not city, state, I'm throwing away the data

tmp = [loc.split(',') for loc in location]
del_idx = [];

for i in range(len(tmp)):
    x = tmp[i]
    try:
        x[1]
    except:
        del_idx.append(i)
    
    
print del_idx

for i in del_idx:
    print i
    del job_links[i]
    del location[i]
    del company_name[i]
    

# for each location, split it by the comma. Then the first (0th) element is the city name
city = [loc.split(',')[0] for loc in location]

# The second (1th) element is the state. This could have garbage, so we just get the first 2 letters which is the
# state code
state = [loc.split(',')[1].strip()[0:2] for loc in location]


   

# print "We found " + str(len(job_links)) + " jobs"
# # We get more because of ads
# print location
# print "There are " + str(len(location)) + " locations (sanity check)"
# print company_name
# print "There are " + str(len(company_name)) + " companies (sanity check)"


# For another time, try to get the job titles
#job_titles_this_page = [str(jp.find("a","jobtitle turnstileLink").get_text()) for jp in job_postings];
#     for jt in job_titles_this_page:
#         job_title.append(jt)


Scraped page 1
Scraped page 2
Scraped page 3
Scraped page 4
Scraped page 5
Scraped page 6
Scraped page 7
Scraped page 8
Scraped page 9
Scraped page 10
Scraped page 11
Scraped page 12
Scraped page 13
Scraped page 14
Scraped page 15
[81, 92, 109, 141]
81
92
109
141


IndexError: list index out of range

In [385]:
tmp = [loc.split(',') for loc in location]
del_idx = [];

for i in range(len(tmp)):
    x = tmp[i]
    try:
        x[1]
    except:
        del_idx.append(i)
del_idx
location[del_idx[1]]

'Remote'

In [ ]:
skill_set = {'mapreduce': 0, 'spark': 0, 'hadoop': 0, 'R': 0, 'python': 0}

# Scrap work

In [128]:
tmp = job_link_area.findAll("div")
not tmp[5].get('class') is None
tmp2 = tmp[6].findAll('span')
#location = [loc for loc in tmp2 if not loc.get('class') is None and ''.join(loc.get('class'))== "location"]
location = tmp[6].findAll("span","location")
#tmp3 = location[0]
print str(location[0].get_text())
print type(str(location[0].get_text()))
str(rem_nl(job_postings[0].find("span",{"company","a"}).get_text())).strip()
str(job_postings[1].find("a","jobtitle turnstileLink").get_text())

Springfield, VA
<type 'str'>


In [250]:
test = {"Company Name": company_name, "City": city, "State": state}
test = pd.DataFrame(test)
test

,City,Company Name,State
0,San Diego,CoreLogic,CA
1,Las Vegas,"Zappos.com, Inc.",NV
2,Santa Barbara,Impact Radius,CA
3,Santa Monica,Westside Analytics,CA
4,Omaha,Gallup,NE
5,Redmond,Microsoft,WA
6,San Francisco,Macy's,CA
7,Palo Alto,Tesla Motors,CA
8,Louisville,Path2Response,CO
9,Washington,DigitalGlobe,DC


In [227]:
location

['San Francisco, CA',
 'Las Vegas, NV',
 'Santa Barbara, CA',
 'Santa Monica, CA',
 'Omaha, NE',
 'Redmond, WA 98052',
 'New York, NY 10032 (Washington Heights area)',
 'San Francisco, CA',
 'Woodland Hills, CA 91367',
 'Palo Alto, CA',
 'Louisville, CO 80027',
 'Mountain View, CA',
 'Irvine, CA',
 'Santa Clara Valley, CA',
 'Des Moines, IA',
 'Redwood City, CA',
 'Bellevue, WA',
 'New York, NY',
 'San Francisco, CA 94103 (South Of Market area)',
 'Atlanta, GA',
 'Green Bay, WI 54301',
 'St. Louis, MO',
 'Piscataway, NJ',
 'Seattle, WA']

In [248]:
tmp = location[6].split(',')
print tmp
print tmp[0]
print type(tmp[0])
tmpnew = tmp[1].strip()[0:2]
print tmpnew
print type(tmpnew)

['New York', ' NY 10032 (Washington Heights area)']
New York
<type 'str'>
NY
<type 'str'>


In [231]:
type(location[0])

str

In [319]:
tmp = job_link_area.findAll("div")
#print tmp[0]
tmp2 = job_link_area.findAll("div",{'class':re.compile('row result'|' row result')})
print len(tmp2)

TypeError: unsupported operand type(s) for |: 'str' and 'str'

In [286]:
tmp3 = [jp for jp in tmp if not jp.get('class') is None 
                     and ''.join(jp.get('class')) =="rowresult"]

In [287]:
len(tmp3)

12

In [305]:
len(tmp2)

9

In [315]:
[rs.get("class") for rs in tmp3]

[['row', 'result'],
 ['', 'row', 'result'],
 ['', 'row', 'result'],
 ['', 'row', 'result'],
 ['', 'row', 'result'],
 ['', 'row', 'result'],
 ['', 'row', 'result'],
 ['', 'row', 'result'],
 ['', 'row', 'result'],
 ['', 'row', 'result'],
 ['row', 'result'],
 ['row', 'result']]

In [322]:
test = {"Company Name": company_name, "Location": location}
test = pd.DataFrame(test)
test


,Company Name,Location
0,Workday,"San Francisco, CA"
1,Picarro,"Santa Clara, CA"
2,Impact Radius,"Santa Barbara, CA"
3,Westside Analytics,"Santa Monica, CA"
4,Gallup,"Omaha, NE"
5,Microsoft,"Redmond, WA 98052"
6,Macy's,"San Francisco, CA"
7,Tesla Motors,"Palo Alto, CA"
8,Path2Response,"Louisville, CO 80027"
9,DigitalGlobe,"Washington, DC"


In [349]:
for i in xrange(len(location)):
    if location[i] == 'Remote':
        location[i] = 'Remote, NS'
    elif location[i] == 'United States':
        location[i] = 'USA, NS'
tmp = [loc.split(',') for loc in location]
tmp2 = [i[0] for i in tmp]
tmp3 = [i[1] for i in tmp]

In [333]:
location

['San Francisco, CA',
 'Santa Clara, CA',
 'Santa Barbara, CA',
 'Santa Monica, CA',
 'Omaha, NE',
 'Redmond, WA 98052',
 'San Francisco, CA',
 'Palo Alto, CA',
 'Louisville, CO 80027',
 'Washington, DC',
 'San Diego, CA',
 'Dearborn, MI',
 'Woonsocket, RI',
 'Santa Clara Valley, CA',
 'Des Moines, IA',
 'Redwood City, CA',
 'Bellevue, WA',
 'Atlanta, GA',
 'San Francisco, CA 94103 (South Of Market area)',
 'Green Bay, WI 54301',
 'St. Louis, MO',
 'San Diego, CA 92121',
 'San Francisco, CA',
 'Highland, UT',
 'Huntsville, AL',
 'Dallas, TX',
 'Princeton, NJ 08540',
 'Bethesda, MD',
 'Palo Alto, CA',
 'Santa Monica, CA',
 'Bellevue, WA 98004 (Downtown area)',
 'Franklin, TN',
 'Blue Bell, PA 19422',
 'Santa Barbara, CA',
 'Broomfield, CO',
 'Los Angeles, CA',
 'Boston, MA',
 'Santa Barbara, CA',
 'San Francisco, CA 94111 (Financial District area)',
 'Santa Barbara, CA',
 'Woodland Hills, CA 91367',
 'Washington, DC',
 'Boston, MA 02109 (Central area)',
 'Boston, MA',
 'Boston, MA',
 'H

In [334]:
tmp

[['San Francisco', ' CA'],
 ['Santa Clara', ' CA'],
 ['Santa Barbara', ' CA'],
 ['Santa Monica', ' CA'],
 ['Omaha', ' NE'],
 ['Redmond', ' WA 98052'],
 ['San Francisco', ' CA'],
 ['Palo Alto', ' CA'],
 ['Louisville', ' CO 80027'],
 ['Washington', ' DC'],
 ['San Diego', ' CA'],
 ['Dearborn', ' MI'],
 ['Woonsocket', ' RI'],
 ['Santa Clara Valley', ' CA'],
 ['Des Moines', ' IA'],
 ['Redwood City', ' CA'],
 ['Bellevue', ' WA'],
 ['Atlanta', ' GA'],
 ['San Francisco', ' CA 94103 (South Of Market area)'],
 ['Green Bay', ' WI 54301'],
 ['St. Louis', ' MO'],
 ['San Diego', ' CA 92121'],
 ['San Francisco', ' CA'],
 ['Highland', ' UT'],
 ['Huntsville', ' AL'],
 ['Dallas', ' TX'],
 ['Princeton', ' NJ 08540'],
 ['Bethesda', ' MD'],
 ['Palo Alto', ' CA'],
 ['Santa Monica', ' CA'],
 ['Bellevue', ' WA 98004 (Downtown area)'],
 ['Franklin', ' TN'],
 ['Blue Bell', ' PA 19422'],
 ['Santa Barbara', ' CA'],
 ['Broomfield', ' CO'],
 ['Los Angeles', ' CA'],
 ['Boston', ' MA'],
 ['Santa Barbara', ' CA'],
 ['

In [359]:
tmp = [loc.split(',') for loc in location]
del_idx = [];

for i in range(len(tmp)):
    x = tmp[i]
    try:
        x[1]
    except:
        print str(i) + " : " + str(x)
        del_idx.append(i)
        

55 : ['Remote']
63 : ['Remote']
71 : ['United States']
81 : ['United States']
99 : ['Remote']
196 : ['United States']
231 : ['United States']
299 : ['United States']
300 : ['Remote']


In [377]:
job_ids

['bcbe8dfea1a8ac59',
 '11ce68b6e69d94f0',
 '66f532481e9722cd',
 '5dcc25778a0548be',
 '8058cdf8282189df',
 'd8357e0227c5f1bf',
 '67366582046d5899',
 '562b7199daf79b7d',
 '2f0dbdd7136614ab',
 '306b329004ab7a9a',
 'fee921890c1d0df8',
 '815b2e991bdd8538']

In [364]:
tmp = [jp.findAll("span") for jp in job_postings];
tmp2 =  [jp for jp in tmp if not jp.get('class') is None 
                     and ''.join(jp.get('class')) =="location"]

AttributeError: 'ResultSet' object has no attribute 'get'

In [370]:
tmp[0].findAll('class'=='location')

AttributeError: 'ResultSet' object has no attribute 'findAll'